To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth your local device, follow [our guide](https://docs.unsloth.ai/get-started/install-and-update). This notebook is licensed [LGPL-3.0](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Long-Context GRPO for reinforcement learning — train stably at massive sequence lengths. Fine-tune models with up to 7x more context length efficiently. [Read Blog](https://unsloth.ai/docs/new/grpo-long-context)

3× faster training with optimized sequence packing — higher throughput with no quality loss.[Read Blog](https://unsloth.ai/docs/new/3x-faster-training-packing)

500k context-length fine-tuning — push long-context models further with memory-efficient training. [Read Blog](https://unsloth.ai/docs/new/500k-context-length-fine-tuning)

Introducing FP8 precision training for faster RL inference. [Read Blog](https://docs.unsloth.ai/new/fp8-reinforcement-learning).

Unsloth's [Docker image](https://hub.docker.com/r/unsloth/unsloth) is here! Start training with no setup & environment issues. [Read our Guide](https://docs.unsloth.ai/new/how-to-train-llms-with-unsloth-and-docker).

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [ ]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

### Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2026.1.4: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Alpaca.ipynb)

For text completions like novel writing, try this [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Mistral_(7B)-Text_Completion.ipynb).

In [ ]:
# PoliERP için Basit Test Verisi Oluşturuyoruz
from datasets import Dataset

erp_data = [
    # =========================
    # GENEL - YÖNLENDİRME
    # =========================
    {
        "instruction": "egemen ERP sisteminde stok durumu nasıl kontrol edilir?",
        "input": "",
        "output": "egemen ERP'de stok durumu için 'Stok Modülü > Güncel Durum' ekranını kullanabilirsin. Alternatif olarak stok kartı üzerinde 'Depo/Lokasyon kırılımı' ile anlık miktarları görebilirsin. İstersen bana 'STK-001 kodlu ürün hangi depolarda var?' gibi sor."
    },
    {
        "instruction": "egemen ERP’de 'Genel Tanımlar' ile 'Stok Modülü' arasındaki fark nedir?",
        "input": "",
        "output": "Genel Tanımlar; birden fazla modülün ortak kullandığı master veriler (Birim, Renk, Malzeme Grubu, Ürün Tipi gibi) içindir. Stok Modülü ise depo/lokasyon, stok kartı, hareketler, lot/seri ve sayım gibi stok yaşam döngüsünü yönetir."
    },
    {
        "instruction": "Depo & Lokasyon nerede konumlanmalı: stok modülü mü genel tanımlar mı?",
        "input": "",
        "output": "Depo ve Lokasyon genelde Stok Modülü altında konumlanır çünkü stok hareketleri, sayım, transfer ve rezervasyon gibi süreçlerin çekirdeğidir. Ancak diğer modüller (Üretim, Sevkiyat) depo seçimi yapacaksa yine ortak kullanılabilir; UI'da Stok Modülü altında, domain olarak 'Inventory' bounded context'te tutmak en sağlıklısı."
    },

    # =========================
    # AUTH / RBAC / PERMISSIONS
    # =========================
    {
        "instruction": "egemen ERP’de login nasıl çalışır? Token nerede saklanır?",
        "input": "",
        "output": "Frontend login olunca backend'den JWT token alır ve localStorage authStorage içine yazar. RTK Query baseApi her isteğe Bearer token ekler. 401 gelirse otomatik logout yapılır ve kullanıcı login sayfasına yönlenir."
    },
    {
        "instruction": "Kullanıcıya rol nasıl atanır?",
        "input": "UserId=... RoleId=...",
        "output": "Backend tarafında 'POST /api/Roles/assign' endpoint'i ile kullanıcıya rol atanır. UI'da bu işlem genelde 'Kullanıcı Detay > Roller' sekmesinden yapılır. Rol atandıktan sonra kullanıcı izinleri rol-permission ilişkisinden hesaplanır."
    },
    {
        "instruction": "Bir role birden fazla permission nasıl atanır?",
        "input": "",
        "output": "Backend'de 'POST /api/RolePermissions/assign' endpoint'i ile role permissionIds listesi gönderilerek atanır. Bu işlem sonrası role bağlı kullanıcıların efektif izinleri değişir; UI tarafında cache invalidation ile role-permissions ve users-permissions yeniden çekilmelidir."
    },
    {
        "instruction": "Efektif izin nedir? Kullanıcı izinleri nasıl hesaplanır?",
        "input": "",
        "output": "Efektif izin; kullanıcının rollerinden gelen izinler + kullanıcıya direkt atanan izinlerin birleşimidir. egemen ERP'de UI menüleri ve butonları bu efektif izin listesine göre görünür/gizli çalışır."
    },
    {
        "instruction": "UI Permission Mapping ne işe yarar?",
        "input": "",
        "output": "UI Permission Mapping; 'pageKey' ve opsiyonel 'buttonKey' ile bir 'permissionCode' eşlemesidir. Böylece frontend'de sayfa ve aksiyonların hangi izne bağlı olduğu yönetilebilir; izin kodları değişse bile UI tarafında standard bir kontrol yapılır."
    },
    {
        "instruction": "403 Forbidden alıyorum, sebebi ne olabilir?",
        "input": "Endpoint: /api/Products",
        "output": "403 genelde yetki eksikliğidir. Kullanıcının rolüne veya user-permission'ına ilgili permissionCode atanmadığı için egemen cy/AuthorizePermission kontrolünden geçemiyor olabilir. Önce kullanıcının efektif izinlerini kontrol et, sonra ilgili role permission ata."
    },

    # =========================
    # PRODUCTS (CRUD) - UI/RTK QUERY
    # =========================
    {
        "instruction": "egemen ERP’de ürün listesi nasıl çekilir?",
        "input": "",
        "output": "Frontend RTK Query üzerinden 'GET /api/Products' çağrısı yapılır. Liste ekranı Syncfusion Grid ile serverdan gelen listeyi gösterir; toolbar search ve actions kolonuyla (edit/delete) yönetilir."
    },
    {
        "instruction": "Ürün güncellemede sık yapılan hata nedir?",
        "input": "",
        "output": "PUT işlemlerinde id’nin hem URL’de hem request body’de gönderilmemesi sık hatadır. egemen ERP standartına göre update çağrısında {id} hem route parametresi hem de body içinde olmalıdır."
    },
    {
        "instruction": "Create/Edit dialog tek ekranda nasıl yönetilir?",
        "input": "",
        "output": "Tek dialog yaklaşımında react-hook-form + zod kullanılır. 'selectedRow' varsa edit mode, yoksa create mode çalışır. Submit işleminde createProduct veya updateProduct mutation seçilir; başarılı olursa grid refresh (invalidateTags) tetiklenir."
    },

    # =========================
    # MASTER DATA (GENEL TANIMLAR)
    # =========================
    {
        "instruction": "Birimler (Units) neden genel tanımlarda tutulur?",
        "input": "",
        "output": "Birimler birçok modül tarafından ortak kullanılır: ürün tanıtımı, stok hareketleri, satınalma, satış, üretim reçetesi. Bu yüzden master data olarak Genel Tanımlar altında yönetilmesi doğru olur."
    },
    {
        "instruction": "Ürün Tipi (ProductType) ve Malzeme Grubu (MaterialGroup) neden önemli?",
        "input": "",
        "output": "Ürün Tipi ürün sınıflandırmasını (hammadde/yarı mamul/mamul vb.) standartlar. Malzeme grubu ise raporlama, fiyatlandırma ve depo stratejileri için gruplama sağlar. İkisi de filtreleme ve raporlama performansını ciddi artırır."
    },

    # =========================
    # INVENTORY / WAREHOUSE / LOCATION
    # =========================
    {
        "instruction": "Lokasyon hiyerarşisini nasıl kurgularız?",
        "input": "",
        "output": "Tipik yapı: Depo > Bölge (Zone) > Raf (Aisle) > Göz (Bin). Lokasyon kod standardı belirlemek (örn: D1-Z01-R03-G07) arama, sayım ve mobil el terminali kullanımını kolaylaştırır."
    },
    {
        "instruction": "Stok kartı açarken minimum hangi alanlar zorunlu olmalı?",
        "input": "",
        "output": "Minimum: Kod, Ad, Ürün Tipi, Birim, (opsiyonel) Malzeme Grubu. Daha sonra süreç olgunlaştıkça lot/seri takibi, kritik stok seviyesi, KDV/hesap kodu gibi alanlar eklenebilir."
    },
    {
        "instruction": "Stok hareketi türleri neler olmalı?",
        "input": "",
        "output": "Temel hareket tipleri: Giriş (satınalma/üretimden giriş), Çıkış (satış/üretime sarf), Transfer (depo/lokasyon arası), Sayım düzeltme (artı/eksi), İade. Her hareket için belge no, tarih, depo/lokasyon, miktar ve açıklama tutulmalıdır."
    },

    # =========================
    # HATA / TROUBLESHOOTING
    # =========================
    {
        "instruction": "Backend’e istek atıyorum ama 401 dönüyor, neye bakmalıyım?",
        "input": "",
        "output": "Token süresi dolmuş olabilir veya request header'a Authorization: Bearer <token> eklenmiyor olabilir. Frontend baseApi prepareHeaders içinde token okuma çalışıyor mu kontrol et. Ayrıca backend tarafında Auth konfigürasyonunda issuer/audience/secret uyumunu doğrula."
    },
    {
        "instruction": "CORS hatası alıyorum (frontend -> backend).",
        "input": "",
        "output": "Backend CORS egemen cy’de frontend origin'i whitelist’e eklenmeli. Dev ortamda Vite origin (örn: http://localhost:5173) izinli değilse tarayıcı bloklar. Ayrıca credentials kullanıyorsan allowCredentials ayarı da gerekli olur."
    },

    # =========================
    # RAPORLAMA / AI RAPOR
    # =========================
    {
        "instruction": "egemen ERP’de yapay zekalı rapor ne demek, örnek ver.",
        "input": "",
        "output": "AI rapor; kullanıcı doğal dille soru sorar (örn: 'Son 7 günde en çok tüketilen malzemeler hangileri?') sistem uygun filtreleri/metric’leri seçip raporu üretir. İlk aşamada hazır rapor şablonlarını (stok yaşlandırma, depo doluluk, en çok giriş/çıkış) LLM ile yönlendirmek pratik olur."
    },

    # =========================
    # DONANIM / TERMINAL (SENİN PROJE BAĞLAMI)
    # =========================

    {
        "instruction": "El terminali / kiosk için lokasyon seçimi UI’da nasıl olmalı?",
        "input": "",
        "output": "Mobil/terminal UI’da lokasyon seçimi için arama + hiyerarşik dropdown mantığı idealdir: önce Depo seç, sonra Zone/Raf/Göz filtrelenerek gelsin. Barkod/QR okutma varsa lokasyon kodunu okutunca otomatik seçilmeli."
    },
]


# Veriyi HuggingFace dataset formatına çeviriyoruz
dataset = Dataset.from_list(erp_data)

# Prompt formatını ayarlayan fonksiyon (Alpaca formatı)
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Cümlenin bittiğini anlatan token

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

# Veriyi map ediyoruz
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/22 [00:00<?, ? examples/s]

<a name="Train"></a>
### Train the model
Now let's train our model. We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    packing = False, # Can make training 5x faster for short sequences.
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 200,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use TrackIO/WandB etc
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/22 [00:00<?, ? examples/s]

🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
9.479 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 22 | Num Epochs = 20 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 29,884,416 of 3,850,963,968 (0.78% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.116100
2,2.075900
3,2.399700
4,2.122300
5,1.952900
6,1.871800
7,1.862600
8,1.884100
9,1.569100
10,1.630800


In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

162.3137 seconds used for training.
2.71 minutes used for training.
Peak reserved memory = 9.479 GB.
Peak reserved memory for training = 0.0 GB.
Peak reserved memory % of max memory = 64.304 %.
Peak reserved memory for training % of max memory = 0.0 %.


<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!



In [ ]:
FastLanguageModel.for_inference(model) # Modeli hızlandır

inputs = tokenizer(
[
    alpaca_prompt.format(
        "Egemen sisteminde stok durumuna nasıl girilir?", # Instruction
        "", # Input
        "", # Output (Boş bırakıyoruz, model dolduracak)
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

["Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nPoliERP sisteminde stok durumuna nasıl girilir?\n\n### Input:\n\n\n### Response:\nPoliERP'de stok durumu için 'Stok Modülü > Güncel Durum' ekranını kullanabilirsin. Alternatif olarak stok kartı üzerinde 'Depo/Lokasyon kırıl"]

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Stok modülünde “Depo Transfer” ekranını göremiyorum ama backend’de transfer endpoint’i çalışıyor. Bu ekranın görünmesi için hangi permission’lar atanmalı, nereden kontrol etmeliyim ve UI’da hangi mapping etkiler?", # Instruction
        "", # Input
        "", # Output (Boş bırakıyoruz, model dolduracak)
    )

], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Stok modülünde “Depo Transfer” ekranını göremiyorum ama backend’de transfer endpoint’i çalışıyor. Bu ekranın görünmesi için hangi permission’lar atanmalı, nereden kontrol etmeliyim ve UI’da hangi mapping etkiler?

### Input:


### Response:
Backend’de kullanıcının rolünü (role) ve izinleri (permissions) ile Authorization grubu (RBAC) tarafında izin kontrol et. UI’da ‘Depo Transfer’ ekranına yönlenirken policy/permission’ı ‘DEPOSIT TRANSFER’ ile UI mapping’ı atıyı teşhisteklerine uymamamam.<|endoftext|>


<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    alpaca_prompt.format(
        "What is a famous tall tower in Paris?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

You can also use Hugging Face's `AutoModelForPeftCausalLM`. Only use this if you do not have `unsloth` installed. It can be hopelessly slow, since `4bit` model downloading is not supported, and Unsloth's **inference is 2x faster**.

In [ ]:
if False:
    # I highly do NOT suggest - use Unsloth if possible
    from peft import AutoPeftModelForCausalLM
    from transformers import AutoTokenizer
    model = AutoPeftModelForCausalLM.from_pretrained(
        "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        load_in_4bit = load_in_4bit,
    )
    tokenizer = AutoTokenizer.from_pretrained("lora_model")

### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False:
    model.push_to_hub("hf/model", token = "")
    tokenizer.push_to_hub("hf/model", token = "")


### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

In [ ]:
model.save_pretrained_gguf("polierp_model", tokenizer, quantization_method = "q4_k_m")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp.

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️

  This notebook and all Unsloth notebooks are licensed [LGPL-3.0](https://github.com/unslothai/notebooks?tab=LGPL-3.0-1-ov-file#readme).
</div>
